In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
%%time
!pip install optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 4.4 MB/s eta 0:00:00a 0:00:01
CPU times: user 34.8 ms, sys: 7.14 ms, total: 41.9 ms
Wall time: 4.64 s


In [2]:
import torch
import polars as pl 
from transformers import AutoModelForImageTextToText, AutoProcessor
from tqdm import tqdm
import warnings
import gc 
import os
import optimum

warnings.filterwarnings("ignore")
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'  


MODEL_PATH = '/kaggle/input/translategemma/transformers/translategemma-12b-it/1'
INPUT_FILE = "/kaggle/input/dataset/dataset.csv"
OUTPUT_FILE = "predictions_translategemma.csv"
MAX_NEW_TOKENS = 256
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

SRC_LANG_SOUTH = "ar"
TGT_LANG_NORTH = "uz"
TGT_LANG_ENG = "en"

# Оптимизации
BATCH_SIZE = 4  # Обрабатываем несколько текстов одновременно
USE_FLASH_ATTENTION = False  # Отключено - требует flash_attn
USE_BETTERTRANSFORMER = True  # Оптимизированный inference


def clean_cuda():
    if DEVICE == "cuda":
        torch.cuda.empty_cache()
        gc.collect()


def load_model():
    print(f"Loading model from {MODEL_PATH}")
    processor = AutoProcessor.from_pretrained(MODEL_PATH)
    
    model = AutoModelForImageTextToText.from_pretrained(
        MODEL_PATH,
        device_map="auto",
        torch_dtype=torch.float16,  # float16 вместо bfloat16 для ускорения
        low_cpu_mem_usage=True
    )
    model.eval()
    
    # BetterTransformer для ускорения inference (требует optimum)
    if USE_BETTERTRANSFORMER:
        try:
            from optimum.bettertransformer import BetterTransformer
            model = BetterTransformer.transform(model)
            print("✓ BetterTransformer enabled")
        except ImportError:
            print("⚠️  BetterTransformer not available (install: pip install optimum)")
        except Exception as e:
            print(f"⚠️  BetterTransformer not available: {e}")
    
    # Torch optimizations
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    
    print(f"Model loaded on device: {next(model.parameters()).device}")
    
    if DEVICE == "cuda":
        print(f"GPU Memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    
    return processor, model


def translate_batch(texts: list[str], processor, model, src_lang: str, tgt_lang: str) -> list[str]:
    """Батчевый перевод для ускорения"""
    translations = []
    
    try:
        # Подготовка всех сообщений
        all_messages = []
        for text in texts:
            messages = [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "source_lang_code": src_lang,
                            "target_lang_code": tgt_lang,
                            "text": text,
                        }
                    ],
                }
            ]
            all_messages.append(messages)
        
        # Обрабатываем батч
        all_inputs = []
        input_lens = []
        
        for messages in all_messages:
            inputs = processor.apply_chat_template(
                messages, 
                tokenize=True, 
                add_generation_prompt=True, 
                return_dict=True, 
                return_tensors="pt"
            )
            all_inputs.append(inputs)
            input_lens.append(inputs['input_ids'].shape[1])
        
        # Паддинг и объединение в батч
        max_len = max(inp['input_ids'].shape[1] for inp in all_inputs)
        
        batched_input_ids = []
        batched_attention_mask = []
        
        for inputs in all_inputs:
            input_ids = inputs['input_ids'][0]
            attention_mask = inputs['attention_mask'][0]
            
            # Padding слева (важно для decoder-only моделей)
            pad_len = max_len - len(input_ids)
            if pad_len > 0:
                input_ids = torch.cat([
                    torch.full((pad_len,), processor.tokenizer.pad_token_id, dtype=input_ids.dtype),
                    input_ids
                ])
                attention_mask = torch.cat([
                    torch.zeros(pad_len, dtype=attention_mask.dtype),
                    attention_mask
                ])
            
            batched_input_ids.append(input_ids)
            batched_attention_mask.append(attention_mask)
        
        batch_inputs = {
            'input_ids': torch.stack(batched_input_ids).to(model.device),
            'attention_mask': torch.stack(batched_attention_mask).to(model.device)
        }
        
        with torch.inference_mode(), torch.cuda.amp.autocast():
            generation = model.generate(
                **batch_inputs,
                max_new_tokens=MAX_NEW_TOKENS,
                do_sample=False,
                pad_token_id=processor.tokenizer.pad_token_id,
                eos_token_id=processor.tokenizer.eos_token_id,
                use_cache=True
            )
        
        # Декодируем результаты
        for i, gen in enumerate(generation):
            result = gen[input_lens[i]:]
            translation = processor.decode(result, skip_special_tokens=True)
            translations.append(translation)
        
        del batch_inputs, generation, all_inputs
        
    except Exception as e:
        print(f"Batch error: {e}")
        translations = [""] * len(texts)
    
    return translations


def run_inference():
    df = pl.read_csv(INPUT_FILE)
    sources = df["uz_south_source"].to_list()
    print(f"Processing {len(sources)} samples with batch size {BATCH_SIZE}")
    
    processor, model = load_model()
    
    print("Warming up model...")
    _ = translate_batch([sources[0]], processor, model, SRC_LANG_SOUTH, TGT_LANG_NORTH)
    clean_cuda()
    
    print(f'\n[1/3] Normalizing South -> North Uzbek (batch size: {BATCH_SIZE})')
    preds_uzn = []
    
    for i in tqdm(range(0, len(sources), BATCH_SIZE), desc="Normalizing", ncols=100):
        batch = sources[i:i+BATCH_SIZE]
        translations = translate_batch(batch, processor, model, SRC_LANG_SOUTH, TGT_LANG_NORTH)
        preds_uzn.extend(translations)
        
        if (i + BATCH_SIZE) % 100 == 0:
            clean_cuda()
    
    clean_cuda()
    
    print(f'\n[2/3] Direct translation South -> English (batch size: {BATCH_SIZE})')
    preds_eng_direct = []
    
    for i in tqdm(range(0, len(sources), BATCH_SIZE), desc="Direct", ncols=100):
        batch = sources[i:i+BATCH_SIZE]
        translations = translate_batch(batch, processor, model, SRC_LANG_SOUTH, TGT_LANG_ENG)
        preds_eng_direct.extend(translations)
        
        if (i + BATCH_SIZE) % 100 == 0:
            clean_cuda()
    
    clean_cuda()
    
    print(f'\n[3/3] Pivot translation North -> English (batch size: {BATCH_SIZE})')
    preds_eng_pivot = []
    
    for i in tqdm(range(0, len(preds_uzn), BATCH_SIZE), desc="Pivot", ncols=100):
        batch = preds_uzn[i:i+BATCH_SIZE]
        translations = translate_batch(batch, processor, model, TGT_LANG_NORTH, TGT_LANG_ENG)
        preds_eng_pivot.extend(translations)
        
        if (i + BATCH_SIZE) % 100 == 0:
            clean_cuda()
    
    clean_cuda()
    
    print("\nSaving results...")
    result_df = df.with_columns([
        pl.Series(name="translategemma_uz_to_uz", values=preds_uzn),
        pl.Series(name="translategemma_uz_to_en_direct", values=preds_eng_direct),
        pl.Series(name="translategemma_uz_to_en_pivot", values=preds_eng_pivot)
    ])
    
    result_df.write_csv(OUTPUT_FILE)
    print(f"✅ Done! Saved to {OUTPUT_FILE}")


if __name__ == "__main__":
    run_inference()

2026-01-17 22:10:01.005978: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768687801.423995     106 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768687801.530021     106 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768687802.496153     106 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768687802.496180     106 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768687802.496183     106 computation_placer.cc:177] computation placer alr

Processing 2579 samples with batch size 4
Loading model from /kaggle/input/translategemma/transformers/translategemma-12b-it/1


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...


[2/3] Direct translation South -> English (batch size: 4)


Direct: 100%|███████████████████████████████████████████████████| 645/645 [2:08:25<00:00, 11.95s/it]



[3/3] Pivot translation North -> English (batch size: 4)


Pivot: 100%|████████████████████████████████████████████████████| 645/645 [2:05:47<00:00, 11.70s/it]



Saving results...
✅ Done! Saved to predictions_translategemma.csv


In [3]:
df = pl.read_csv('/kaggle/working/predictions_translategemma.csv').to_pandas()

In [4]:
df

,id,en_source,uz_south_source,uz_north_synthetic_ref,split,translategemma_uz_to_uz,translategemma_uz_to_en_direct,translategemma_uz_to_en_pivot
0,1861,"Meanwhile, Mohammad Talib Shahid, head of publ...",شونده ی بیرحالده، هرات ساغلیق نی سقلش رئیسی مح...,Ayni paytda Hirotdagi sog'liqni saqlash boshqa...,validation,هرات ده پولیو گه قرشی ایملش اوچون برچه تیارگر...,هرات ده پولیو گه قرشی ایملش اوچون برچه تیارگر...,قرشی ایملش اوچون برچه تیارگرلیک آلینگن دیر.\n...
1,59,This is because foreigners pursue their own go...,مقاومت مردمی اۉزیگه ته یه نیشی و تۉلیق مستقللی...,Chunki xorijliklar har qanday holatda ham o‘z ...,validation,explanations or commentary. Please translate ...,explanations or commentary. Please translate ...,
2,1308,According to the provincial governor's media o...,ولایت مطبوعات بۉلیمی نینگ خبریگه کۉره ، او ولا...,Viloyat gubernatori matbuot xizmati xabariga k...,validation,,,convey the meaning and nuances of the origina...
3,1480,She also called for the financial cooperation ...,شونینگدیک او، یاردم بیرووچی نهاد لردن بدخشان د...,"U, shuningdek, Badaxshondagi qizlar va ayollar...",validation,یاردم بیرووچی نهاد لردن بدخشان ده گی قیزلر و ...,یاردم بیرووچی نهاد لردن بدخشان ده گی قیزلر و ...,سته گن.\nmodel\nmodel\n
4,2323,"""The child has been sick for 20 days.","""کسل بولگنی دن ۲۰ کون بوله دی.",“Bola 20 kundan beri kasal.,validation,user\nYou are a professional Arabic (ar) to Uz...,user\nYou are a professional Arabic (ar) to En...,
...,...,...,...,...,...,...,...,...
2574,822,"Simultaneously, a source from the institute in...",عین حالده، بو انستیتوتده گی بیر منبع نینگ ۸صبح...,"Shu bilan birga, institutdagi manbaning Hasht-...",test,model\n,model\n,"grammar, vocabulary, and cultural sensitiviti..."
2575,1032,"In 2023, Transparency International assessed a...",شفّافلیک تشکیلاتی ۲۰۲۳- ییلده دنیا نینگ ۱۸۰ ته...,2023 yilda Transparency International dunyo bo...,test,۲۳- ییلده دنیا نینگ ۱۸۰ ته دولتیده مأموری شفّا...,۲۳- ییلده دنیا نینگ ۱۸۰ ته دولتیده مأموری شفّا...,
2576,902,Richard Bennett Urges Pakistan to Halt the Dep...,ریچارد بنت ینه بیر بار پاکستان دن افغانستانلیک...,Richard Bennet Pokistonni afg‘on qochqinlarini...,test,,,user\nYou are a professional Uzbek (uz) to Eng...
2577,2423,According to the statistics provided by the mi...,اوشبو وزیرلیک آرقه لی ارائه بولگن آمار اساسیده...,Vazirlik tomonidan taqdim etilgan statistik ma...,test,، بو سوو تاشقین لرده قریب ۱۵ مینگ یشش اوی بوتو...,، بو سوو تاشقین لرده قریب ۱۵ مینگ یشش اوی بوتو...,لی و قسمی شکلده ویران بولیب و آلتی مینگ دن آرت...


In [6]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [9]:
import evaluate
bert_score = evaluate.load('bert_score')

FileNotFoundError: Couldn't find a module script at /kaggle/working/bert_score/bert_score.py. Module 'bert_score' doesn't exist on the Hugging Face Hub either.

In [11]:
!pip install bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.6 MB/s eta 0:00:00


In [15]:
import polars as pl
import evaluate
from tqdm import tqdm
import numpy as np


print("Загружаем BERTScore...")
bert_score = evaluate.load('bertscore')

# Извлекаем данные
references = df["uz_north_synthetic_ref"].to_list()  # Эталонные переводы
predictions = df["translategemma_uz_to_uz"].to_list()  # Предсказания модели

print(f"\nВсего примеров: {len(references)}")
print(f"Пример эталона: {references[0][:100]}...")
print(f"Пример предсказания: {predictions[0][:100]}...")

# Вариант 1: Оценка всего датасета сразу (может занять много памяти)
# print("\n" + "="*60)
# print("ВАРИАНТ 1: Полная оценка")
# print("="*60)

# results = bert_score.compute(
#     predictions=predictions,
#     references=references,
#     lang="uz",  # Язык: узбекский
#     model_type="xlm-roberta-base",  # Многоязычная модель
#     device="cuda"  # Используем GPU
# )

# print(f"\n📊 Результаты BERTScore:")
# print(f"  Precision: {np.mean(results['precision']):.4f}")
# print(f"  Recall:    {np.mean(results['recall']):.4f}")
# print(f"  F1:        {np.mean(results['f1']):.4f}")


# Вариант 2: Батчевая оценка (для больших датасетов)
print("\n" + "="*60)
print("ВАРИАНТ 2: Батчевая оценка (экономия памяти)")
print("="*60)

batch_size = 64
all_precision = []
all_recall = []
all_f1 = []

for i in tqdm(range(0, len(predictions), batch_size), desc="Evaluating"):
    batch_preds = predictions[i:i+batch_size]
    batch_refs = references[i:i+batch_size]
    
    batch_results = bert_score.compute(
        predictions=batch_preds,
        references=batch_refs,
        lang="uz",
        model_type="xlm-roberta-base",
        device="cuda"
    )
    
    all_precision.extend(batch_results['precision'])
    all_recall.extend(batch_results['recall'])
    all_f1.extend(batch_results['f1'])

print(f"\n📊 Батчевые результаты BERTScore:")
print(f"  Precision: {np.mean(all_precision):.4f} ± {np.std(all_precision):.4f}")
print(f"  Recall:    {np.mean(all_recall):.4f} ± {np.std(all_recall):.4f}")
print(f"  F1:        {np.mean(all_f1):.4f} ± {np.std(all_f1):.4f}")


# Вариант 3: Сравнение разных переводов
print("\n" + "="*60)
print("ВАРИАНТ 3: Сравнение прямого и pivot переводов")
print("="*60)

# Direct translation (uz_south → en)
direct_refs = df["en_source"].to_list()
direct_preds = df["translategemma_uz_to_en_direct"].to_list()

# Pivot translation (uz_south → uz_north → en)
pivot_preds = df["translategemma_uz_to_en_pivot"].to_list()

print("\nОценка прямого перевода (uz_south → en)...")
direct_results = bert_score.compute(
    predictions=direct_preds,
    references=direct_refs,
    lang="en",
    model_type="microsoft/deberta-xlarge-mnli",  # Лучше для английского
    device="cuda"
)

print("\nОценка pivot перевода (uz_south → uz_north → en)...")
pivot_results = bert_score.compute(
    predictions=pivot_preds,
    references=direct_refs,
    lang="en",
    model_type="microsoft/deberta-xlarge-mnli",
    device="cuda"
)

print(f"\n📊 Сравнение методов перевода:")
print(f"\nПрямой перевод (uz_south → en):")
print(f"  Precision: {np.mean(direct_results['precision']):.4f}")
print(f"  Recall:    {np.mean(direct_results['recall']):.4f}")
print(f"  F1:        {np.mean(direct_results['f1']):.4f}")

print(f"\nPivot перевод (uz_south → uz_north → en):")
print(f"  Precision: {np.mean(pivot_results['precision']):.4f}")
print(f"  Recall:    {np.mean(pivot_results['recall']):.4f}")
print(f"  F1:        {np.mean(pivot_results['f1']):.4f}")

improvement = np.mean(direct_results['f1']) - np.mean(pivot_results['f1'])
print(f"\n{'✅' if improvement > 0 else '❌'} Прямой перевод {'лучше' if improvement > 0 else 'хуже'} на {abs(improvement):.4f} F1")


# Вариант 4: Анализ по примерам
print("\n" + "="*60)
print("ВАРИАНТ 4: Топ-5 лучших и худших переводов")
print("="*60)

# Считаем F1 для каждого примера
individual_scores = bert_score.compute(
    predictions=predictions,
    references=references,
    lang="uz",
    model_type="xlm-roberta-base",
    device="cuda"
)

f1_scores = individual_scores['f1']

# Топ-5 лучших
best_indices = np.argsort(f1_scores)[-5:][::-1]
print("\n🏆 Топ-5 лучших переводов:")
for idx in best_indices:
    print(f"\nF1: {f1_scores[idx]:.4f}")
    print(f"  Эталон:     {references[idx][:80]}...")
    print(f"  Предсказание: {predictions[idx][:80]}...")

# Топ-5 худших
worst_indices = np.argsort(f1_scores)[:5]
print("\n\n⚠️  Топ-5 худших переводов:")
for idx in worst_indices:
    print(f"\nF1: {f1_scores[idx]:.4f}")
    print(f"  Эталон:     {references[idx][:80]}...")
    print(f"  Предсказание: {predictions[idx][:80]}...")


# Сохраняем результаты
print("\n" + "="*60)
print("Сохранение результатов...")
print("="*60)

df_with_scores = df.with_columns([
    pl.Series(name="bertscore_precision", values=individual_scores['precision']),
    pl.Series(name="bertscore_recall", values=individual_scores['recall']),
    pl.Series(name="bertscore_f1", values=individual_scores['f1'])
])

df_with_scores.write_csv("predictions_with_bertscore.csv")
print("✅ Сохранено в predictions_with_bertscore.csv")

print("\n📈 Общая статистика:")
print(f"  Средний F1: {np.mean(f1_scores):.4f}")
print(f"  Медиана F1: {np.median(f1_scores):.4f}")
print(f"  Min F1: {np.min(f1_scores):.4f}")
print(f"  Max F1: {np.max(f1_scores):.4f}")

Загружаем BERTScore...

Всего примеров: 2579
Пример эталона: Ayni paytda Hirotdagi sog'liqni saqlash boshqarmasi boshlig'i Muhammad Tolib Shahid Hirotda poliomiy...
Пример предсказания:  هرات ده پولیو گه قرشی ایملش اوچون برچه تیارگرلیک آلینگن دیر.
model
...

ВАРИАНТ 2: Батчевая оценка (экономия памяти)


Evaluating:   0%|          | 0/41 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/3.04G [00:00<?, ?B/s]


Оценка pivot перевода (uz_south → uz_north → en)...



📊 Сравнение методов перевода:

Прямой перевод (uz_south → en):
  Precision: 0.2136
  Recall:    0.2895
  F1:        0.2410

Pivot перевод (uz_south → uz_north → en):
  Precision: 0.2627
  Recall:    0.2917
  F1:        0.2737

❌ Прямой перевод хуже на 0.0327 F1

ВАРИАНТ 4: Топ-5 лучших и худших переводов



🏆 Топ-5 лучших переводов:

F1: 0.8359
  Эталон:     Abdul Jabbor ismli yana bir haydovchi: “Yo‘l juda yomon....
  Предсказание:  بیری عبدالجبار گه کوره: "یول جوده هم بوزوق دیر.
model
...

F1: 0.8272
  Эталон:     Ushbu fojiali voqea 17-dekabr, yakshanba kuni Islomobodning “G-9/4” hududida yuz...
  Предсказание: حادثه ۲۶- قوس یک شنبه کونی اسلام آباد شهری نینگ "G-9/4"منطقه سیده یوز بېرگن.
mod...

F1: 0.8243
  Эталон:     Ayollar huquqlari faollari va ayrim siyosiy faollar Afg‘onistondagi BMTni Tolibo...
  Предсказание:  یوناما را "یک جانبه و دور از واقعیت" خوانده بود.
model
...

F1: 0.8231
  Эталон:     Biroq tashkilot ushbu kampaniya davomida qaysi viloyatlarda qancha bola vaktsina...
  Предсказание: ی حقیده هېچ نرسه دیمه گن.
model
...

F1: 0.8224
  Эталон:     Statistik ma’lumotlarga ko‘ra, ushbu dala va issiqxonalardan 66 tonna bodring ol...
  Предсказание:  مذکور ایکینزار لر و گل خانه لردن، ۶۶ تُن بادرنگ توپلنگن.
model
...


⚠️  Топ-5 худших переводов:

F1: 0.0000
  Эталон:     Balx

AttributeError: 'DataFrame' object has no attribute 'with_columns'

In [14]:
df.columns

Index(['id', 'en_source', 'uz_south_source', 'uz_north_synthetic_ref', 'split',
       'translategemma_uz_to_uz', 'translategemma_uz_to_en_direct',
       'translategemma_uz_to_en_pivot'],
      dtype='object')

In [18]:
pred1 = pl.read_csv('/kaggle/input/predict1/predictions_model_1.csv').to_pandas()
pred1.sample(10)

,id,en_source,uz_south_source,uz_north_synthetic_ref,split,nllb_uz_to_uz,nllb_uz_to_en_direct,nllb_uz_to_en_pivot,nllb_uz_to_en_ensemble
1515,1138,Afghanistan to Face Qatar in First Match of FI...,جهان قهرمانلیگی جامی و فوتبال بوییچه آسیا ملتل...,FIFA Jahon chempionati va Osiyo Kubogi saralas...,test,Jahon chempionligi kubogi va futbol bo'limi Os...,world championship cup and the football bowl A...,World Cup and soccer division in battle with N...,World Cup and soccer division in battle with N...
1061,1134,According to the head of the Third Political D...,تشقی ایشلر وزیرلیگی نینگ اوچینچی سیاسی باشقرمه...,Tashqi ishlar vazirligining uchinchi siyosiy b...,test,Tashqi Ashler Ministrlik Ittifoqi Ning Uchinch...,Investigative Ashler Minister of the League of...,Statements by the Head of the Third Political ...,Statements of the Head of the Third Political ...
1737,897,The Ministry of Europe and Foreign Affairs of ...,اروپا وزیرلیگی و فرانسه تشقی ایشلری نینگ ایتیش...,Fransiyaning Yevropa va tashqi ishlar vazirlig...,test,Yevropa Ittifoqi va Fransiya vaziri Ning Aitic...,Ministers of Europe and France are investigati...,Minister of the European Union and France Ning...,Minister of the European Union and France Ning...
1389,1216,The Organization for Health Administration and...,ساغلیقنی سقلش باشقه ریش و رواجلنتیریش مؤسسه سی...,Sog'liqni saqlashni boshqarish va rivojlantiri...,test,Sog'liqni saqlash bo'yicha tashxis qo'yish va ...,Rashtriya Swayamsevak Sangh and the Rawalpindi...,Institute for Diagnostics and Registry of Heal...,Institute for Diagnostics and Registration in ...
1295,130,Taliban Arrest Five in Takhar Province for Pla...,طالبلر تخارده بېش کیشینی موسیقه چالگنلری اوچون...,Tolibon Taxar viloyatida musiqa chalayotgan be...,test,"Talabalar""Besh Keshini""musiqa qo'ng'iroqlari u...",Students are invited to take part in the Besh ...,"Students enter the""Oshla Gunler""class at 8 a. ...","Students enter the""Oshla Gunler""class at 8 a. ..."
2067,79,Despite sustained efforts to combat polio thro...,بو شونده ی حالده که، افغانستان ده پولیو کوره ش...,Emlash kampaniyalari orqali poliomielitga qars...,test,"Shunga qaramay, Afg'onistondagi poliolovka kas...",This is despite the fact that polio in Afghani...,"And yet, the polio epidemic in Afghanistan is ...","And yet, the polio epidemic in Afghanistan is ..."
1834,2088,"These goods are of good quality.""","مذکور توارلر یخشی کیفیت گه ایگه دیر.""",Bu mahsulotlar sifatli”.,test,Yuqorida aytib o'tilgan sovutgichlarning muzla...,aforementioned cooling tower is of very high q...,freezing quality of the aforementioned refrige...,quality of the above-mentioned refrigerants is...
1384,706,They say they do not feel safe even in the onl...,بیر لقمه نان تاپیش اوچون آنلاین فروشگاهلرنی ای...,Ular hatto onlayn muhitda ham o'zlarini xavfsi...,test,Internet do'konlarida non to'plamini topish uc...,online store for a loaf of bread was invented ...,Created by Etish Algurgon to find a bun in onl...,It was created by Etish Algurganer to find bre...
2238,1698,"According to him, the Pakistani Taliban are us...",اونگه کوره، پاکستان لیک طالب لر، پاکستان گه هج...,"Uning so‘zlariga ko‘ra, Pokiston toliblari Afg...",test,"Oʻng Kore, Pokistonlik Talablar va Pokistonlik...","Ongh Kore, Pakistan's leaked Taliban, Pakistan...","Right-Korean, Pakistani Students and Pakistani...","Right-Korean, Pakistani Students and Pakistani..."
1996,1959,The deputy of publishing affairs of the Minist...,معلومات و مدنیت وزیرلیگی نینگ نشرات ایشلری بوی...,Axborot va madaniyat vazirligi nashriyot ishla...,test,Ma'lumot va madaniyat vazirligi Ning nashriyot...,Minister of Information and Civilization of th...,Ministry of Information and Culture's Publicat...,Ministry of Information and Culture's Publicat...


In [20]:
import polars as pl
import evaluate
from tqdm import tqdm
import numpy as np

df = pred1


print("Загружаем BERTScore...")
bert_score = evaluate.load('bertscore')

# Извлекаем данные
references = df["uz_north_synthetic_ref"].to_list()  # Эталонные переводы
predictions = df["nllb_uz_to_uz"].to_list()  # Предсказания модели

print(f"\nВсего примеров: {len(references)}")
print(f"Пример эталона: {references[0][:100]}...")
print(f"Пример предсказания: {predictions[0][:100]}...")

# Вариант 1: Оценка всего датасета сразу (может занять много памяти)
# print("\n" + "="*60)
# print("ВАРИАНТ 1: Полная оценка")
# print("="*60)

# results = bert_score.compute(
#     predictions=predictions,
#     references=references,
#     lang="uz",  # Язык: узбекский
#     model_type="xlm-roberta-base",  # Многоязычная модель
#     device="cuda"  # Используем GPU
# )

# print(f"\n📊 Результаты BERTScore:")
# print(f"  Precision: {np.mean(results['precision']):.4f}")
# print(f"  Recall:    {np.mean(results['recall']):.4f}")
# print(f"  F1:        {np.mean(results['f1']):.4f}")


# Вариант 2: Батчевая оценка (для больших датасетов)
print("\n" + "="*60)
print("ВАРИАНТ 2: Батчевая оценка (экономия памяти)")
print("="*60)

batch_size = 64
all_precision = []
all_recall = []
all_f1 = []

for i in tqdm(range(0, len(predictions), batch_size), desc="Evaluating"):
    batch_preds = predictions[i:i+batch_size]
    batch_refs = references[i:i+batch_size]
    
    batch_results = bert_score.compute(
        predictions=batch_preds,
        references=batch_refs,
        lang="uz",
        model_type="xlm-roberta-base",
        device="cuda"
    )
    
    all_precision.extend(batch_results['precision'])
    all_recall.extend(batch_results['recall'])
    all_f1.extend(batch_results['f1'])

print(f"\n📊 Батчевые результаты BERTScore:")
print(f"  Precision: {np.mean(all_precision):.4f} ± {np.std(all_precision):.4f}")
print(f"  Recall:    {np.mean(all_recall):.4f} ± {np.std(all_recall):.4f}")
print(f"  F1:        {np.mean(all_f1):.4f} ± {np.std(all_f1):.4f}")


# Вариант 3: Сравнение разных переводов
print("\n" + "="*60)
print("ВАРИАНТ 3: Сравнение прямого и pivot переводов")
print("="*60)

# Direct translation (uz_south → en)
direct_refs = df["en_source"].to_list()
direct_preds = df["nllb_uz_to_en_direct"].to_list()

# Pivot translation (uz_south → uz_north → en)
pivot_preds = df["nllb_uz_to_en_pivot"].to_list()

print("\nОценка прямого перевода (uz_south → en)...")
direct_results = bert_score.compute(
    predictions=direct_preds,
    references=direct_refs,
    lang="en",
    model_type="microsoft/deberta-xlarge-mnli",  # Лучше для английского
    device="cuda"
)

print("\nОценка pivot перевода (uz_south → uz_north → en)...")
pivot_results = bert_score.compute(
    predictions=pivot_preds,
    references=direct_refs,
    lang="en",
    model_type="microsoft/deberta-xlarge-mnli",
    device="cuda"
)

print(f"\n📊 Сравнение методов перевода:")
print(f"\nПрямой перевод (uz_south → en):")
print(f"  Precision: {np.mean(direct_results['precision']):.4f}")
print(f"  Recall:    {np.mean(direct_results['recall']):.4f}")
print(f"  F1:        {np.mean(direct_results['f1']):.4f}")

print(f"\nPivot перевод (uz_south → uz_north → en):")
print(f"  Precision: {np.mean(pivot_results['precision']):.4f}")
print(f"  Recall:    {np.mean(pivot_results['recall']):.4f}")
print(f"  F1:        {np.mean(pivot_results['f1']):.4f}")

improvement = np.mean(direct_results['f1']) - np.mean(pivot_results['f1'])
print(f"\n{'✅' if improvement > 0 else '❌'} Прямой перевод {'лучше' if improvement > 0 else 'хуже'} на {abs(improvement):.4f} F1")


# Вариант 4: Анализ по примерам
print("\n" + "="*60)
print("ВАРИАНТ 4: Топ-5 лучших и худших переводов")
print("="*60)

# Считаем F1 для каждого примера
individual_scores = bert_score.compute(
    predictions=predictions,
    references=references,
    lang="uz",
    model_type="xlm-roberta-base",
    device="cuda"
)

f1_scores = individual_scores['f1']

# Топ-5 лучших
best_indices = np.argsort(f1_scores)[-5:][::-1]
print("\n🏆 Топ-5 лучших переводов:")
for idx in best_indices:
    print(f"\nF1: {f1_scores[idx]:.4f}")
    print(f"  Эталон:     {references[idx][:80]}...")
    print(f"  Предсказание: {predictions[idx][:80]}...")

# Топ-5 худших
worst_indices = np.argsort(f1_scores)[:5]
print("\n\n⚠️  Топ-5 худших переводов:")
for idx in worst_indices:
    print(f"\nF1: {f1_scores[idx]:.4f}")
    print(f"  Эталон:     {references[idx][:80]}...")
    print(f"  Предсказание: {predictions[idx][:80]}...")



print("\n📈 Общая статистика:")
print(f"  Средний F1: {np.mean(f1_scores):.4f}")
print(f"  Медиана F1: {np.median(f1_scores):.4f}")
print(f"  Min F1: {np.min(f1_scores):.4f}")
print(f"  Max F1: {np.max(f1_scores):.4f}")

Загружаем BERTScore...

Всего примеров: 2579
Пример эталона: Ayni paytda Hirotdagi sog'liqni saqlash boshqarmasi boshlig'i Muhammad Tolib Shahid Hirotda poliomiy...
Пример предсказания: Shu bilan birga, Heratning sog'liqni saqlash bo'yicha tashvishlari rahbari Muhammad Talab Shahid Nin...

ВАРИАНТ 2: Батчевая оценка (экономия памяти)


Evaluating: 100%|██████████| 41/41 [00:03<00:00, 12.00it/s]



📊 Батчевые результаты BERTScore:
  Precision: 0.8607 ± 0.0231
  Recall:    0.8702 ± 0.0213
  F1:        0.8653 ± 0.0207

ВАРИАНТ 3: Сравнение прямого и pivot переводов

Оценка прямого перевода (uz_south → en)...

Оценка pivot перевода (uz_south → uz_north → en)...

📊 Сравнение методов перевода:

Прямой перевод (uz_south → en):
  Precision: 0.5750
  Recall:    0.6276
  F1:        0.5989

Pivot перевод (uz_south → uz_north → en):
  Precision: 0.6280
  Recall:    0.6511
  F1:        0.6383

❌ Прямой перевод хуже на 0.0394 F1

ВАРИАНТ 4: Топ-5 лучших и худших переводов

🏆 Топ-5 лучших переводов:

F1: 1.0000
  Эталон:     Dori yo'q....
  Предсказание: Dori yo'q....

F1: 0.9552
  Эталон:     Mazkur yig‘ilishda O‘zbekiston, Xitoy, Eron, Pokiston, Rossiya, Tojikiston va Tu...
  Предсказание: Uchrashuvda O'zbekiston, Xitoy, Iran, Pokiston, Rossiya, Tojikiston va Turkmanis...

F1: 0.9489
  Эталон:     Guterrishning so‘zlariga ko‘ra, jamoa 16,8 million kishidan 12,5 millionini eng ...
  Предсказ

In [21]:
old_pred1 = pl.read_csv('/kaggle/input/oldpred1/predictions_model_1_old.csv').to_pandas()
old_pred1.sample(5)

,id,en_source,uz_south_source,uz_north_synthetic_ref,split,nllb_uz_to_en,nllb_uz_to_uz
1691,1682,No person or group has yet claimed responsibil...,اوشبو هجوم مسئولیتینی حاضرگچه هیچ کیم یا گروه ...,Hozircha hech kim yoki guruh bu hujum uchun ja...,test,Oshbu did not take responsibility for the atta...,Oshbo hujumining mas'uliyatini hech qanday kim...
1720,991,The international organization has called for ...,بو خلق ارا تشکیلات اسرائیل ریجیمی رسميلریدن تو...,Xalqaro tashkilot Isroil rasmiylarini tushunti...,test,After the creation of the newly formed Israel ...,Isroil rasmiy rejimining rasmiy tuzilishini ta...
451,2112,"According to them, a number of children were s...",اولر گه کوره، قطار باله لر عایله لری دن اجره ت...,"Ularning so‘zlariga ko‘ra, bir qancha bolalar ...",validation,"Oller Ghorh, a ballet dancer, and his family w...",O'shanda ko'pchilik oila a'zolari mehnatga haq...
636,2357,"Akhtar Mohammad, a resident of Giru district o...",غزنی نینگ گیرو تومنی ده گی مذکور باله لر یقین ...,G‘aznining Giru tumanida yashovchi Axtar Muham...,test,Ghazni's Ning Giro Toomni will be mentioned by...,"G'azni ning g'iro tomni aytiladi: ""G'iro tomni..."
2280,1181,"""The Islamic Emirate of Afghanistan will not a...","مجاهد تأکیدله دی: ""افغانستان اسلام امیرلیگی هی...",“Afgʻoniston Islom Amirligi hech kimga afgʻon ...,test,"Mujahid emphasized: ""The Islamic Emirate of Af...","Mujahid ta'kidlab o'tdi: ""Afg'oniston Islom Am..."


In [23]:
import polars as pl
import evaluate
from tqdm import tqdm
import numpy as np

df = old_pred1


print("Загружаем BERTScore...")
bert_score = evaluate.load('bertscore')

# Извлекаем данные
references = df["uz_north_synthetic_ref"].to_list()  # Эталонные переводы
predictions = df["nllb_uz_to_uz"].to_list()  # Предсказания модели

print(f"\nВсего примеров: {len(references)}")
print(f"Пример эталона: {references[0][:100]}...")
print(f"Пример предсказания: {predictions[0][:100]}...")

# Вариант 1: Оценка всего датасета сразу (может занять много памяти)
# print("\n" + "="*60)
# print("ВАРИАНТ 1: Полная оценка")
# print("="*60)

# results = bert_score.compute(
#     predictions=predictions,
#     references=references,
#     lang="uz",  # Язык: узбекский
#     model_type="xlm-roberta-base",  # Многоязычная модель
#     device="cuda"  # Используем GPU
# )

# print(f"\n📊 Результаты BERTScore:")
# print(f"  Precision: {np.mean(results['precision']):.4f}")
# print(f"  Recall:    {np.mean(results['recall']):.4f}")
# print(f"  F1:        {np.mean(results['f1']):.4f}")


# Вариант 2: Батчевая оценка (для больших датасетов)
print("\n" + "="*60)
print("ВАРИАНТ 2: Батчевая оценка (экономия памяти)")
print("="*60)

batch_size = 64
all_precision = []
all_recall = []
all_f1 = []

for i in tqdm(range(0, len(predictions), batch_size), desc="Evaluating"):
    batch_preds = predictions[i:i+batch_size]
    batch_refs = references[i:i+batch_size]
    
    batch_results = bert_score.compute(
        predictions=batch_preds,
        references=batch_refs,
        lang="uz",
        model_type="xlm-roberta-base",
        device="cuda"
    )
    
    all_precision.extend(batch_results['precision'])
    all_recall.extend(batch_results['recall'])
    all_f1.extend(batch_results['f1'])

print(f"\n📊 Батчевые результаты BERTScore:")
print(f"  Precision: {np.mean(all_precision):.4f} ± {np.std(all_precision):.4f}")
print(f"  Recall:    {np.mean(all_recall):.4f} ± {np.std(all_recall):.4f}")
print(f"  F1:        {np.mean(all_f1):.4f} ± {np.std(all_f1):.4f}")


# Вариант 3: Сравнение разных переводов
print("\n" + "="*60)
print("ВАРИАНТ 3: Сравнение прямого и pivot переводов")
print("="*60)

# Direct translation (uz_south → en)
direct_refs = df["en_source"].to_list()
direct_preds = df["nllb_uz_to_en"].to_list()

# Pivot translation (uz_south → uz_north → en)
# pivot_preds = df["nllb_uz_to_en_pivot"].to_list()

print("\nОценка прямого перевода (uz_south → en)...")
direct_results = bert_score.compute(
    predictions=direct_preds,
    references=direct_refs,
    lang="en",
    model_type="microsoft/deberta-xlarge-mnli",  # Лучше для английского
    device="cuda"
)

# print("\nОценка pivot перевода (uz_south → uz_north → en)...")
# pivot_results = bert_score.compute(
#     predictions=pivot_preds,
#     references=direct_refs,
#     lang="en",
#     model_type="microsoft/deberta-xlarge-mnli",
#     device="cuda"
# )

print(f"\n📊 Сравнение методов перевода:")
print(f"\nПрямой перевод (uz_south → en):")
print(f"  Precision: {np.mean(direct_results['precision']):.4f}")
print(f"  Recall:    {np.mean(direct_results['recall']):.4f}")
print(f"  F1:        {np.mean(direct_results['f1']):.4f}")

# print(f"\nPivot перевод (uz_south → uz_north → en):")
# print(f"  Precision: {np.mean(pivot_results['precision']):.4f}")
# print(f"  Recall:    {np.mean(pivot_results['recall']):.4f}")
# print(f"  F1:        {np.mean(pivot_results['f1']):.4f}")

# improvement = np.mean(direct_results['f1']) - np.mean(pivot_results['f1'])
# print(f"\n{'✅' if improvement > 0 else '❌'} Прямой перевод {'лучше' if improvement > 0 else 'хуже'} на {abs(improvement):.4f} F1")


# Вариант 4: Анализ по примерам
print("\n" + "="*60)
print("ВАРИАНТ 4: Топ-5 лучших и худших переводов")
print("="*60)

# Считаем F1 для каждого примера
individual_scores = bert_score.compute(
    predictions=predictions,
    references=references,
    lang="uz",
    model_type="xlm-roberta-base",
    device="cuda"
)

f1_scores = individual_scores['f1']

# Топ-5 лучших
best_indices = np.argsort(f1_scores)[-5:][::-1]
print("\n🏆 Топ-5 лучших переводов:")
for idx in best_indices:
    print(f"\nF1: {f1_scores[idx]:.4f}")
    print(f"  Эталон:     {references[idx][:80]}...")
    print(f"  Предсказание: {predictions[idx][:80]}...")

# Топ-5 худших
worst_indices = np.argsort(f1_scores)[:5]
print("\n\n⚠️  Топ-5 худших переводов:")
for idx in worst_indices:
    print(f"\nF1: {f1_scores[idx]:.4f}")
    print(f"  Эталон:     {references[idx][:80]}...")
    print(f"  Предсказание: {predictions[idx][:80]}...")



print("\n📈 Общая статистика:")
print(f"  Средний F1: {np.mean(f1_scores):.4f}")
print(f"  Медиана F1: {np.median(f1_scores):.4f}")
print(f"  Min F1: {np.min(f1_scores):.4f}")
print(f"  Max F1: {np.max(f1_scores):.4f}")

Загружаем BERTScore...

Всего примеров: 2579
Пример эталона: Ayni paytda Hirotdagi sog'liqni saqlash boshqarmasi boshlig'i Muhammad Tolib Shahid Hirotda poliomiy...
Пример предсказания: Shu bilan birga, Heratning sog'liqni saqlash bo'yicha tashvishlar bo'yicha bosh direktori Muhammad T...

ВАРИАНТ 2: Батчевая оценка (экономия памяти)


Evaluating: 100%|██████████| 41/41 [00:03<00:00, 10.68it/s]



📊 Батчевые результаты BERTScore:
  Precision: 0.8585 ± 0.0292
  Recall:    0.8675 ± 0.0256
  F1:        0.8629 ± 0.0260

ВАРИАНТ 3: Сравнение прямого и pivot переводов

Оценка прямого перевода (uz_south → en)...

📊 Сравнение методов перевода:

Прямой перевод (uz_south → en):
  Precision: 0.5686
  Recall:    0.6249
  F1:        0.5939

ВАРИАНТ 4: Топ-5 лучших и худших переводов

🏆 Топ-5 лучших переводов:

F1: 1.0000
  Эталон:     Dori yo'q....
  Предсказание: Dori yo'q....

F1: 0.9552
  Эталон:     Mazkur yig‘ilishda O‘zbekiston, Xitoy, Eron, Pokiston, Rossiya, Tojikiston va Tu...
  Предсказание: Uchrashuvda O'zbekiston, Xitoy, Iran, Pokiston, Rossiya, Tojikiston va Turkmanis...

F1: 0.9489
  Эталон:     Guterrishning so‘zlariga ko‘ra, jamoa 16,8 million kishidan 12,5 millionini eng ...
  Предсказание: Guterresning so'zlariga ko'ra, ushbu guruh 16,8 million kishidan 12,5 million ki...

F1: 0.9413
  Эталон:     U UNAMA o'z hisobotlarini nashr etishdan oldin o'z vazirligi bilan bo'lishis

In [24]:
pred2 = pl.read_csv('/kaggle/input/pred2/predictions_model_2.csv').to_pandas()
pred2.sample(5)

,id,en_source,uz_south_source,uz_north_synthetic_ref,split,pred_eng_m2m,pred_uzn_m2m
2575,1032,"In 2023, Transparency International assessed a...",شفّافلیک تشکیلاتی ۲۰۲۳- ییلده دنیا نینگ ۱۸۰ ته...,2023 yilda Transparency International dunyo bo...,test,The 2023-Year-Old World NING 180 was governed ...,2023- Yildiz World NING 180-da qiyari qiyari q...
346,301,"These units, along with outpatient mental heal...",بو بۉلیملر روان ساغلیک کلینیکلری ایاق اوستیده ...,"Ushbu bo'linmalar, ambulator ruhiy salomatlik ...",validation,B.B.L.M. Psychiatric Clinic is a psychiatric s...,B.B.Limler psikiatlik kliniklar bilan psikiatl...
654,276,Sources: Taliban Yet to Pay Two Months of Gove...,منبعلر: طالبلرهنوز دولت ایشلاوچیلری نینگ ایکی ...,Manbalar: Tolibon hukumat xodimlarining ikki o...,test,Publisher: ThalblrHanus Government Ishlauchill...,Mumbai: TullblarHanishishishishishishishishish...
391,809,"As a result of this attack, two Shia clerics a...",هجوم نتیجه سیده شیعه مذهبلی ایکّی دین عالم قۉش...,Ushbu hujum natijasida ikki shia ulamosi va to...,validation,"As a result of the attack, the Syiah religious...",Uqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq...
1183,1454,"In this meeting, the Acting Minister of Defens...",مدافعه وزیرلیگی سرپرستی اوشبو اوچره شووده آمری...,Ushbu uchrashuvda Mudofaa vaziri vazifasini ba...,test,Defense Ministry Supervisory Oshbo Ochery said...,Uşbo Uşre Uşre Uşre Uşre Uşre Uşre Uşre Uşre U...


In [25]:
import polars as pl
import evaluate
from tqdm import tqdm
import numpy as np

df = pred2


print("Загружаем BERTScore...")
bert_score = evaluate.load('bertscore')

# Извлекаем данные
references = df["uz_north_synthetic_ref"].to_list()  # Эталонные переводы
predictions = df["pred_uzn_m2m"].to_list()  # Предсказания модели

print(f"\nВсего примеров: {len(references)}")
print(f"Пример эталона: {references[0][:100]}...")
print(f"Пример предсказания: {predictions[0][:100]}...")

# Вариант 1: Оценка всего датасета сразу (может занять много памяти)
# print("\n" + "="*60)
# print("ВАРИАНТ 1: Полная оценка")
# print("="*60)

# results = bert_score.compute(
#     predictions=predictions,
#     references=references,
#     lang="uz",  # Язык: узбекский
#     model_type="xlm-roberta-base",  # Многоязычная модель
#     device="cuda"  # Используем GPU
# )

# print(f"\n📊 Результаты BERTScore:")
# print(f"  Precision: {np.mean(results['precision']):.4f}")
# print(f"  Recall:    {np.mean(results['recall']):.4f}")
# print(f"  F1:        {np.mean(results['f1']):.4f}")


# Вариант 2: Батчевая оценка (для больших датасетов)
print("\n" + "="*60)
print("ВАРИАНТ 2: Батчевая оценка (экономия памяти)")
print("="*60)

batch_size = 64
all_precision = []
all_recall = []
all_f1 = []

for i in tqdm(range(0, len(predictions), batch_size), desc="Evaluating"):
    batch_preds = predictions[i:i+batch_size]
    batch_refs = references[i:i+batch_size]
    
    batch_results = bert_score.compute(
        predictions=batch_preds,
        references=batch_refs,
        lang="uz",
        model_type="xlm-roberta-base",
        device="cuda"
    )
    
    all_precision.extend(batch_results['precision'])
    all_recall.extend(batch_results['recall'])
    all_f1.extend(batch_results['f1'])

print(f"\n📊 Батчевые результаты BERTScore:")
print(f"  Precision: {np.mean(all_precision):.4f} ± {np.std(all_precision):.4f}")
print(f"  Recall:    {np.mean(all_recall):.4f} ± {np.std(all_recall):.4f}")
print(f"  F1:        {np.mean(all_f1):.4f} ± {np.std(all_f1):.4f}")


# Вариант 3: Сравнение разных переводов
print("\n" + "="*60)
print("ВАРИАНТ 3: Сравнение прямого и pivot переводов")
print("="*60)

# Direct translation (uz_south → en)
direct_refs = df["en_source"].to_list()
direct_preds = df["pred_eng_m2m"].to_list()

# Pivot translation (uz_south → uz_north → en)
# pivot_preds = df["nllb_uz_to_en_pivot"].to_list()

print("\nОценка прямого перевода (uz_south → en)...")
direct_results = bert_score.compute(
    predictions=direct_preds,
    references=direct_refs,
    lang="en",
    model_type="microsoft/deberta-xlarge-mnli",  # Лучше для английского
    device="cuda"
)

# print("\nОценка pivot перевода (uz_south → uz_north → en)...")
# pivot_results = bert_score.compute(
#     predictions=pivot_preds,
#     references=direct_refs,
#     lang="en",
#     model_type="microsoft/deberta-xlarge-mnli",
#     device="cuda"
# )

print(f"\n📊 Сравнение методов перевода:")
print(f"\nПрямой перевод (uz_south → en):")
print(f"  Precision: {np.mean(direct_results['precision']):.4f}")
print(f"  Recall:    {np.mean(direct_results['recall']):.4f}")
print(f"  F1:        {np.mean(direct_results['f1']):.4f}")

# print(f"\nPivot перевод (uz_south → uz_north → en):")
# print(f"  Precision: {np.mean(pivot_results['precision']):.4f}")
# print(f"  Recall:    {np.mean(pivot_results['recall']):.4f}")
# print(f"  F1:        {np.mean(pivot_results['f1']):.4f}")

# improvement = np.mean(direct_results['f1']) - np.mean(pivot_results['f1'])
# print(f"\n{'✅' if improvement > 0 else '❌'} Прямой перевод {'лучше' if improvement > 0 else 'хуже'} на {abs(improvement):.4f} F1")


# Вариант 4: Анализ по примерам
print("\n" + "="*60)
print("ВАРИАНТ 4: Топ-5 лучших и худших переводов")
print("="*60)

# Считаем F1 для каждого примера
individual_scores = bert_score.compute(
    predictions=predictions,
    references=references,
    lang="uz",
    model_type="xlm-roberta-base",
    device="cuda"
)

f1_scores = individual_scores['f1']

# Топ-5 лучших
best_indices = np.argsort(f1_scores)[-5:][::-1]
print("\n🏆 Топ-5 лучших переводов:")
for idx in best_indices:
    print(f"\nF1: {f1_scores[idx]:.4f}")
    print(f"  Эталон:     {references[idx][:80]}...")
    print(f"  Предсказание: {predictions[idx][:80]}...")

# Топ-5 худших
worst_indices = np.argsort(f1_scores)[:5]
print("\n\n⚠️  Топ-5 худших переводов:")
for idx in worst_indices:
    print(f"\nF1: {f1_scores[idx]:.4f}")
    print(f"  Эталон:     {references[idx][:80]}...")
    print(f"  Предсказание: {predictions[idx][:80]}...")



print("\n📈 Общая статистика:")
print(f"  Средний F1: {np.mean(f1_scores):.4f}")
print(f"  Медиана F1: {np.median(f1_scores):.4f}")
print(f"  Min F1: {np.min(f1_scores):.4f}")
print(f"  Max F1: {np.max(f1_scores):.4f}")

Загружаем BERTScore...

Всего примеров: 2579
Пример эталона: Ayni paytda Hirotdagi sog'liqni saqlash boshqarmasi boshlig'i Muhammad Tolib Shahid Hirotda poliomiy...
Пример предсказания: U‘zari, u‘zari, u‘zari, u‘zari, u‘zari, u‘zari, u‘zari, u‘zari, u‘zari, u‘zari, u‘zari, u‘zari, u‘za...

ВАРИАНТ 2: Батчевая оценка (экономия памяти)


Evaluating: 100%|██████████| 41/41 [00:03<00:00, 10.99it/s]



📊 Батчевые результаты BERTScore:
  Precision: 0.7323 ± 0.0534
  Recall:    0.7549 ± 0.0442
  F1:        0.7430 ± 0.0464

ВАРИАНТ 3: Сравнение прямого и pivot переводов

Оценка прямого перевода (uz_south → en)...

📊 Сравнение методов перевода:

Прямой перевод (uz_south → en):
  Precision: 0.4185
  Recall:    0.5139
  F1:        0.4580

ВАРИАНТ 4: Топ-5 лучших и худших переводов

🏆 Топ-5 лучших переводов:

F1: 0.8581
  Эталон:     U qo‘shimcha qildi: “Biz hayvonlarni bozorga sotish uchun olib keldik....
  Предсказание: Oning Cup Ytirishik: "Satilish o'qon market o'qiltirgan mas....

F1: 0.8579
  Эталон:     Tolibon ham bu hujumni qoraladi....
  Предсказание: Tullbler o'nun kontinentini saldırmaz....

F1: 0.8578
  Эталон:     “Bola 20 kundan beri kasal....
  Предсказание: "Bulganni day 20 ko'ni bulgandi....

F1: 0.8564
  Эталон:     Bir ishchining kunlik maoshi 300 afg'on, tadbirkorning maoshi 700 dan 800 afg'on...
  Предсказание: İshchi Ning Qonday Leki 300 afgan, 700-800 afgan qonday q

In [28]:
!pip install sacrebleu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 3.1 MB/s eta 0:00:00


,id,en_source,uz_south_source,uz_north_synthetic_ref,split,nllb_uz_to_en,nllb_uz_to_uz
0,1861,"Meanwhile, Mohammad Talib Shahid, head of publ...",شونده ی بیرحالده، هرات ساغلیق نی سقلش رئیسی مح...,Ayni paytda Hirotdagi sog'liqni saqlash boshqa...,validation,"In the meantime, Herat's head of health affair...","Shu bilan birga, Heratning sog'liqni saqlash b..."
1,59,This is because foreigners pursue their own go...,مقاومت مردمی اۉزیگه ته یه نیشی و تۉلیق مستقللی...,Chunki xorijliklar har qanday holatda ham o‘z ...,validation,The resistance of the people to the revolution...,Odamlar qarshilik ko'rsatishga qarshilik ko'rs...
2,1308,According to the provincial governor's media o...,ولایت مطبوعات بۉلیمی نینگ خبریگه کۉره ، او ولا...,Viloyat gubernatori matbuot xizmati xabariga k...,validation,"He was interviewed by the provincial press, an...","O'zbekiston viloyati matbuot xizmati, viloyat ..."
3,1480,She also called for the financial cooperation ...,شونینگدیک او، یاردم بیرووچی نهاد لردن بدخشان د...,"U, shuningdek, Badaxshondagi qizlar va ayollar...",validation,"Shuningdeek O, Yardem Beroochy Institute, has ...","Shuningdek O, Yardam Bayruchi instituti tomoni..."
4,2323,"""The child has been sick for 20 days.","""کسل بولگنی دن ۲۰ کون بوله دی.",“Bola 20 kundan beri kasal.,validation,"""Ksell Bolgny did not bow for 20 days.","""Ksell Bolgunni kuni 20 kun bo'lib o'tdi."
...,...,...,...,...,...,...,...
2574,822,"Simultaneously, a source from the institute in...",عین حالده، بو انستیتوتده گی بیر منبع نینگ ۸صبح...,"Shu bilan birga, institutdagi manbaning Hasht-...",test,"At the same time, Bo Institute has a resource ...","Shu bilan birga, bu institutda bir manba bor b..."
2575,1032,"In 2023, Transparency International assessed a...",شفّافلیک تشکیلاتی ۲۰۲۳- ییلده دنیا نینگ ۱۸۰ ته...,2023 yilda Transparency International dunyo bo...,test,"In 2023, when the world numbered 180, Sharif c...",2023-yilda Jahonning 180 yoshli davlatidagi ma...
2576,902,Richard Bennett Urges Pakistan to Halt the Dep...,ریچارد بنت ینه بیر بار پاکستان دن افغانستانلیک...,Richard Bennet Pokistonni afg‘on qochqinlarini...,test,Richard Bennett once again in Pakistan on the ...,Richard Bennet bir marta Pokiston kunida Afg'o...
2577,2423,According to the statistics provided by the mi...,اوشبو وزیرلیک آرقه لی ارائه بولگن آمار اساسیده...,Vazirlik tomonidan taqdim etilgan statistik ma...,test,Oshbu ministers have been orderly presenting b...,Oshbo vazirining tartib-taomilini ko'rib chiqi...


In [31]:
import polars as pl
import numpy as np
from sacrebleu.metrics import BLEU, CHRF
from bert_score import score as bert_score
import mlflow
import evaluate
import os
import warnings
import torch
from tqdm import tqdm

warnings.filterwarnings('ignore')


FILES = {
    "NLLB-200": "/kaggle/input/predict1/predictions_model_1.csv",
    "M2M100": "/kaggle/input/pred2/predictions_model_2.csv"
}
EXPERIMENT_NAME = "South_Uzbek_Translation_Benchmark"
COMPUTE_BERT = True
BERT_BATCH_SIZE = 32
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


def compute_bleu(predictions, references):
    """BLEU score"""
    refs = [[r] for r in references]
    return BLEU().corpus_score(predictions, refs).score


def compute_chrf(predictions, references):
    """chrF score"""
    refs = [[r] for r in references]
    return CHRF().corpus_score(predictions, refs).score


def compute_meteor(predictions, references):
    """METEOR score"""
    meteor = evaluate.load("meteor")
    clean_preds = [p or "" for p in predictions]
    clean_refs = [r or "" for r in references]
    result = meteor.compute(predictions=clean_preds, references=clean_refs)
    return result["meteor"] * 100


def compute_bertscore(predictions, references, lang='en'):
    """BERTScore с правильной моделью для каждого языка"""
    if not COMPUTE_BERT:
        return {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}
    
    # Очистка данных
    clean_preds = [str(p) if p else "" for p in predictions]
    clean_refs = [str(r) if r else "" for r in references]
    
    # Выбор модели в зависимости от языка
    if lang == 'en':
        model_type = 'microsoft/deberta-xlarge-mnli'
        print(f"    Computing BERTScore (English, {model_type})...")
    else:  # uz или other
        model_type = 'xlm-roberta-base'
        print(f"    Computing BERTScore (Uzbek, {model_type})...")
    
    P, R, F1 = bert_score(
        clean_preds, 
        clean_refs, 
        model_type=model_type,
        lang=lang if lang == 'en' else None,  # Для других языков lang=None
        verbose=False, 
        device=DEVICE, 
        batch_size=BERT_BATCH_SIZE
    )
    
    return {
        'precision': P.mean().item() * 100, 
        'recall': R.mean().item() * 100, 
        'f1': F1.mean().item() * 100
    }


def compute_length_stats(predictions, references):
    """Статистика длин текстов"""
    pred_len = [len(str(p).split()) for p in predictions]
    ref_len = [len(str(r).split()) for r in references]
    return {
        'avg_pred_length': np.mean(pred_len),
        'avg_ref_length': np.mean(ref_len),
        'length_ratio': np.mean(pred_len) / np.mean(ref_len) if np.mean(ref_len) > 0 else 0,
        'std_pred_length': np.std(pred_len),
        'std_ref_length': np.std(ref_len)
    }


def compute_all_metrics(predictions, references, lang='en', desc=""):
    """Вычисляем все метрики"""
    print(f"  Evaluating {desc}...")
    
    results = {
        'bleu': compute_bleu(predictions, references),
        'chrf': compute_chrf(predictions, references),
        'meteor': compute_meteor(predictions, references)
    }
    
    results.update(compute_length_stats(predictions, references))
    
    if COMPUTE_BERT:
        bert = compute_bertscore(predictions, references, lang=lang)
        results.update({
            'bertscore_f1': bert['f1'], 
            'bertscore_precision': bert['precision'], 
            'bertscore_recall': bert['recall']
        })
    
    print(f"    ✓ BLEU: {results['bleu']:.2f} | chrF: {results['chrf']:.2f} | METEOR: {results['meteor']:.2f}", end="")
    if COMPUTE_BERT:
        print(f" | BERTScore F1: {results['bertscore_f1']:.2f}")
    else:
        print()
    
    return results


def evaluate_nllb(df):
    """Оценка NLLB модели"""
    print("\n" + "="*80)
    print("📊 Evaluating NLLB-200")
    print("="*80)
    
    refs_en = df["en_source"].to_list()
    refs_uz = df["uz_north_synthetic_ref"].to_list()
    
    results = {}
    
    # 1. Uzbek normalization (uz_south -> uz_north)
    if "nllb_uz_to_uz" in df.columns:
        print("\n[1/4] Uzbek Normalization (uz_south → uz_north)")
        preds_uz = df["nllb_uz_to_uz"].to_list()
        results['uzbek_normalization'] = compute_all_metrics(
            preds_uz, refs_uz, lang='other', desc="Uzbek Normalization"
        )
    
    # 2. Direct translation (uz_south -> en)
    if "nllb_uz_to_en_direct" in df.columns:
        print("\n[2/4] Direct Translation (uz_south → en)")
        preds_direct = df["nllb_uz_to_en_direct"].to_list()
        results['english_direct'] = compute_all_metrics(
            preds_direct, refs_en, lang='en', desc="Direct Translation"
        )
    
    # 3. Pivot translation (uz_south -> uz_north -> en)
    if "nllb_uz_to_en_pivot" in df.columns:
        print("\n[3/4] Pivot Translation (uz_south → uz_north → en)")
        preds_pivot = df["nllb_uz_to_en_pivot"].to_list()
        results['english_pivot'] = compute_all_metrics(
            preds_pivot, refs_en, lang='en', desc="Pivot Translation"
        )
    
    # 4. Ensemble translation
    if "nllb_uz_to_en_ensemble" in df.columns:
        print("\n[4/4] Ensemble Translation")
        preds_ensemble = df["nllb_uz_to_en_ensemble"].to_list()
        results['english_ensemble'] = compute_all_metrics(
            preds_ensemble, refs_en, lang='en', desc="Ensemble Translation"
        )
    
    # Дополнительно: базовый метод если есть
    if "nllb_uz_to_en" in df.columns:
        print("\n[Extra] Base Translation")
        preds_base = df["nllb_uz_to_en"].to_list()
        results['english_base'] = compute_all_metrics(
            preds_base, refs_en, lang='en', desc="Base Translation"
        )
    
    return results


def evaluate_m2m(df):
    """Оценка M2M100 модели"""
    print("\n" + "="*80)
    print("📊 Evaluating M2M100")
    print("="*80)
    
    refs_en = df["en_source"].to_list()
    refs_uz = df["uz_north_synthetic_ref"].to_list()
    
    results = {}
    
    # 1. Uzbek normalization (uz_south -> uz_north)
    if "pred_uzn_m2m" in df.columns:
        print("\n[1/2] Uzbek Normalization (uz_south → uz_north)")
        preds_uz = df["pred_uzn_m2m"].to_list()
        results['uzbek_normalization'] = compute_all_metrics(
            preds_uz, refs_uz, lang='other', desc="Uzbek Normalization"
        )
    
    # 2. English translation (uz_south -> en)
    if "pred_eng_m2m" in df.columns:
        print("\n[2/2] English Translation (uz_south → en)")
        preds_en = df["pred_eng_m2m"].to_list()
        results['english_translation'] = compute_all_metrics(
            preds_en, refs_en, lang='en', desc="English Translation"
        )
    
    return results


def log_to_mlflow(model_name, results, df, filename):
    """Логируем результаты в MLflow"""
    with mlflow.start_run(run_name=f"Eval_{model_name}"):
        # Параметры
        mlflow.log_param("model", model_name)
        mlflow.log_param("test_samples", df.height)
        mlflow.log_param("prediction_file", filename)
        mlflow.log_param("compute_bertscore", COMPUTE_BERT)
        mlflow.log_param("device", DEVICE)
        
        # Метрики
        for task_name, metrics in results.items():
            prefix = task_name.replace(" ", "_").lower()
            for metric_name, value in metrics.items():
                mlflow.log_metric(f"{prefix}_{metric_name}", value)
        
        # Артефакты
        if os.path.exists(filename):
            mlflow.log_artifact(filename)


def evaluate_model(model_name, filename):
    """Основная функция оценки модели"""
    if not os.path.exists(filename):
        print(f"❌ File not found: {filename}")
        return None
    
    print(f"\n{'='*80}")
    print(f"📂 Loading {filename}")
    print(f"{'='*80}")
    
    df = pl.read_csv(filename)
    
    # Фильтруем validation split если есть
    if "split" in df.columns:
        df = df.filter(pl.col("split") == "validation")
        print(f"✓ Using validation split: {df.height} samples")
    else:
        print(f"✓ Using full dataset: {df.height} samples")
    
    # Оцениваем в зависимости от типа модели
    is_nllb = "nllb" in model_name.lower()
    
    if is_nllb:
        results = evaluate_nllb(df)
    else:
        results = evaluate_m2m(df)
    
    # Логируем в MLflow
    log_to_mlflow(model_name, results, df, filename)
    
    return results


def print_comparison_table(all_results):
    """Печатаем сравнительную таблицу"""
    print("\n" + "="*80)
    print("📊 COMPARISON TABLE")
    print("="*80)
    
    for model_name, results in all_results.items():
        print(f"\n{model_name}:")
        for task, metrics in results.items():
            print(f"  {task}:")
            print(f"    BLEU: {metrics['bleu']:.2f} | chrF: {metrics['chrf']:.2f} | METEOR: {metrics['meteor']:.2f}", end="")
            if 'bertscore_f1' in metrics:
                print(f" | BERTScore F1: {metrics['bertscore_f1']:.2f}")
            else:
                print()


def run_evaluation():
    """Запуск полной оценки"""
    print(f"\n{'='*80}")
    print(f"🚀 Starting Translation Evaluation")
    print(f"{'='*80}")
    print(f"Device: {DEVICE}")
    print(f"BERTScore: {'Enabled' if COMPUTE_BERT else 'Disabled'}")
    print(f"Models to evaluate: {list(FILES.keys())}")
    
    mlflow.set_experiment(EXPERIMENT_NAME)
    
    all_results = {}
    
    for model_name, filename in FILES.items():
        results = evaluate_model(model_name, filename)
        if results:
            all_results[model_name] = results
    
    # Печатаем итоговую таблицу
    print_comparison_table(all_results)
    
    print("\n" + "="*80)
    print("✅ Evaluation completed!")
    print("💡 View results: mlflow ui")
    print("="*80)


if __name__ == "__main__":
    run_evaluation()


🚀 Starting Translation Evaluation
Device: cuda
BERTScore: Enabled
Models to evaluate: ['NLLB-200', 'M2M100']


2026/01/18 05:08:20 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/18 05:08:20 INFO mlflow.store.db.utils: Updating database tables
2026/01/18 05:08:20 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/18 05:08:20 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/01/18 05:08:20 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
2026/01/18 05:08:21 INFO alembic.runtime.migration: Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
2026/01/18 05:08:21 INFO alembic.runtime.migration: Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
2026/01/18 05:08:21 INFO alembic.runtime.migration: Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
2026/01/18 05:08:21 INFO alembic.runtime.migration: Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
2026/01/18 05:08:21 INFO alembic.runtime.migration: Running 


📂 Loading /kaggle/input/predict1/predictions_model_1.csv
✓ Using validation split: 516 samples

📊 Evaluating NLLB-200

[1/4] Uzbek Normalization (uz_south → uz_north)
  Evaluating Uzbek Normalization...


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


    Computing BERTScore (Uzbek, xlm-roberta-base)...
    ✓ BLEU: 14.75 | chrF: 43.84 | METEOR: 16.04 | BERTScore F1: 86.65

[2/4] Direct Translation (uz_south → en)
  Evaluating Direct Translation...


...

    Computing BERTScore (English, microsoft/deberta-xlarge-mnli)...
    ✓ BLEU: 24.33 | chrF: 39.82 | METEOR: 28.94 | BERTScore F1: 64.09

📂 Loading /kaggle/input/pred2/predictions_model_2.csv
✓ Using validation split: 516 samples

📊 Evaluating M2M100

[1/2] Uzbek Normalization (uz_south → uz_north)
  Evaluating Uzbek Normalization...


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


    Computing BERTScore (Uzbek, xlm-roberta-base)...
    ✓ BLEU: 2.71 | chrF: 10.01 | METEOR: 3.04 | BERTScore F1: 74.03

[2/2] English Translation (uz_south → en)
  Evaluating English Translation...


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


    Computing BERTScore (English, microsoft/deberta-xlarge-mnli)...
    ✓ BLEU: 10.28 | chrF: 18.59 | METEOR: 16.80 | BERTScore F1: 46.40

📊 COMPARISON TABLE

NLLB-200:
  uzbek_normalization:
    BLEU: 14.75 | chrF: 43.84 | METEOR: 16.04 | BERTScore F1: 86.65
  english_direct:
    BLEU: 8.80 | chrF: 38.27 | METEOR: 27.10 | BERTScore F1: 60.29
  english_pivot:
    BLEU: 24.46 | chrF: 34.92 | METEOR: 28.74 | BERTScore F1: 64.16
  english_ensemble:
    BLEU: 24.33 | chrF: 39.82 | METEOR: 28.94 | BERTScore F1: 64.09

M2M100:
  uzbek_normalization:
    BLEU: 2.71 | chrF: 10.01 | METEOR: 3.04 | BERTScore F1: 74.03
  english_translation:
    BLEU: 10.28 | chrF: 18.59 | METEOR: 16.80 | BERTScore F1: 46.40

✅ Evaluation completed!
💡 View results: mlflow ui


In [30]:
!pip install mlflow

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 75.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 26.0rc2
    Uninstalling packaging-26.0rc2:
      Successfully uninstalled packaging-26.0rc2
ERROR: pip's dependency resolver

In [32]:
import os
import warnings
from dataclasses import dataclass
from typing import Dict, List

import numpy as np
import polars as pl
import torch
import mlflow
import evaluate
from sacrebleu.metrics import BLEU, CHRF
from bert_score import score as bert_score

warnings.filterwarnings("ignore")

# ================== CONFIG ==================

FILES = {
    "NLLB-200": "/kaggle/input/predict1/predictions_model_1.csv",
    "M2M100": "/kaggle/input/pred2/predictions_model_2.csv"
}

EXPERIMENT_NAME = "South_Uzbek_Translation_Benchmark"
COMPUTE_BERT = True
BERT_BATCH_SIZE = 32
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


# ================== METRICS ==================

def compute_bleu(preds, refs):
    refs = [[r] for r in refs]
    return BLEU().corpus_score(preds, refs).score


def compute_chrf(preds, refs):
    refs = [[r] for r in refs]
    return CHRF().corpus_score(preds, refs).score


def compute_meteor(preds, refs):
    meteor = evaluate.load("meteor")
    preds = [p or "" for p in preds]
    refs = [r or "" for r in refs]
    return meteor.compute(predictions=preds, references=refs)["meteor"] * 100


def compute_length_stats(preds, refs):
    p = [len(str(x).split()) for x in preds]
    r = [len(str(x).split()) for x in refs]
    return {
        "avg_pred_length": np.mean(p),
        "avg_ref_length": np.mean(r),
        "length_ratio": np.mean(p) / max(np.mean(r), 1e-6),
        "std_pred_length": np.std(p),
        "std_ref_length": np.std(r),
    }


# ================== BERTSCORE ==================

def compute_bertscore(preds, refs, lang):
    if not COMPUTE_BERT:
        return {"bertscore_f1": 0.0, "bertscore_precision": 0.0, "bertscore_recall": 0.0}

    preds = [str(p or "") for p in preds]
    refs = [str(r or "") for r in refs]

    if lang == "en":
        model_type = "microsoft/deberta-xlarge-mnli"
    else:
        model_type = "xlm-roberta-base"

    P, R, F1 = bert_score(
        preds,
        refs,
        model_type=model_type,
        lang=lang if lang == "en" else None,
        device=DEVICE,
        batch_size=BERT_BATCH_SIZE,
        verbose=False
    )

    return {
        "bertscore_precision": P.mean().item() * 100,
        "bertscore_recall": R.mean().item() * 100,
        "bertscore_f1": F1.mean().item() * 100,
    }


# ================== CORE EVAL ==================

def evaluate_task(preds, refs, lang, name):
    print(f"  ▶ Evaluating: {name}")

    results = {
        "bleu": compute_bleu(preds, refs),
        "chrf": compute_chrf(preds, refs),
        "meteor": compute_meteor(preds, refs),
    }

    results.update(compute_length_stats(preds, refs))
    results.update(compute_bertscore(preds, refs, lang))

    print(
        f"    BLEU={results['bleu']:.2f} | chrF={results['chrf']:.2f} | "
        f"METEOR={results['meteor']:.2f} | BERT-F1={results['bertscore_f1']:.2f}"
    )

    return results


# ================== MODEL CONFIGS ==================

@dataclass
class TaskConfig:
    pred_col: str
    ref_col: str
    lang: str
    name: str


NLLB_TASKS = [
    TaskConfig("nllb_uz_to_uz", "uz_north_synthetic_ref", "other", "Uzbek normalization"),
    TaskConfig("nllb_uz_to_en_direct", "en_source", "en", "Direct translation"),
    TaskConfig("nllb_uz_to_en_pivot", "en_source", "en", "Pivot translation"),
    TaskConfig("nllb_uz_to_en_ensemble", "en_source", "en", "Ensemble translation"),
]

M2M_TASKS = [
    TaskConfig("pred_uzn_m2m", "uz_north_synthetic_ref", "other", "Uzbek normalization"),
    TaskConfig("pred_eng_m2m", "en_source", "en", "English translation"),
]


# ================== EVALUATION ==================

def evaluate_model(model_name, path):
    print(f"\n📂 Loading: {path}")
    df = pl.read_csv(path)

    if "split" in df.columns:
        df = df.filter(pl.col("split") == "validation")
        print(f"✓ Using validation split: {df.height} samples")

    tasks = NLLB_TASKS if "nllb" in model_name.lower() else M2M_TASKS

    results = {}

    for task in tasks:
        if task.pred_col not in df.columns:
            continue

        preds = df[task.pred_col].to_list()
        refs = df[task.ref_col].to_list()

        results[task.name] = evaluate_task(preds, refs, task.lang, task.name)

    return results, df


# ================== MLFLOW ==================

def log_to_mlflow(model_name, results, df, filename):
    with mlflow.start_run(run_name=f"Eval_{model_name}"):
        mlflow.log_param("model", model_name)
        mlflow.log_param("samples", df.height)
        mlflow.log_param("file", filename)

        for task, metrics in results.items():
            prefix = task.replace(" ", "_").lower()
            for k, v in metrics.items():
                mlflow.log_metric(f"{prefix}_{k}", v)

        mlflow.log_artifact(filename)


# ================== RUN ==================

def main():
    print("🚀 Starting evaluation")
    print("Device:", DEVICE)
    print("BERTScore:", COMPUTE_BERT)

    mlflow.set_experiment(EXPERIMENT_NAME)

    all_results = {}

    for model_name, path in FILES.items():
        results, df = evaluate_model(model_name, path)
        log_to_mlflow(model_name, results, df, path)
        all_results[model_name] = results

    print("\n✅ DONE")


if __name__ == "__main__":
    main()


🚀 Starting evaluation
Device: cuda
BERTScore: True

📂 Loading: /kaggle/input/predict1/predictions_model_1.csv
✓ Using validation split: 516 samples
  ▶ Evaluating: Uzbek normalization


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


    BLEU=14.75 | chrF=43.84 | METEOR=16.04 | BERT-F1=86.65
  ▶ Evaluating: Direct translation


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


    BLEU=8.80 | chrF=38.27 | METEOR=27.10 | BERT-F1=60.29
  ▶ Evaluating: Pivot translation


...

    BLEU=24.33 | chrF=39.82 | METEOR=28.94 | BERT-F1=64.09

📂 Loading: /kaggle/input/pred2/predictions_model_2.csv
✓ Using validation split: 516 samples
  ▶ Evaluating: Uzbek normalization


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


    BLEU=2.71 | chrF=10.01 | METEOR=3.04 | BERT-F1=74.03
  ▶ Evaluating: English translation


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


    BLEU=10.28 | chrF=18.59 | METEOR=16.80 | BERT-F1=46.40

✅ DONE


In [35]:
import warnings
from dataclasses import dataclass

import numpy as np
import polars as pl
import torch
import mlflow
import evaluate
from sacrebleu.metrics import BLEU, CHRF
from bert_score import score as bert_score

warnings.filterwarnings("ignore")

# ================== CONFIG ==================

FILES = {
    "NLLB-200": "/kaggle/input/predict1/predictions_model_1.csv",
    "M2M100": "/kaggle/input/pred2/predictions_model_2.csv"
}

EXPERIMENT_NAME = "South_Uzbek_Translation_Benchmark"
COMPUTE_BERT = True
BERT_BATCH_SIZE = 32
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ================== METRICS ==================

def compute_bleu(preds: list[str], refs: list[str]) -> float:
    refs = [[r] for r in refs]
    return BLEU().corpus_score(preds, refs).score


def compute_chrf(preds: list[str], refs: list[str]) -> float:
    refs = [[r] for r in refs]
    return CHRF().corpus_score(preds, refs).score


def compute_meteor(preds: list[str], refs: list[str]) -> float:
    meteor = evaluate.load("meteor")
    preds = [p or "" for p in preds]
    refs = [r or "" for r in refs]
    return meteor.compute(predictions=preds, references=refs)["meteor"] * 100


def compute_length_stats(preds: list[str], refs: list[str]) -> dict[str, float]:
    p = [len(str(x).split()) for x in preds]
    r = [len(str(x).split()) for x in refs]
    return {
        "avg_pred_length": np.mean(p),
        "avg_ref_length": np.mean(r),
        "length_ratio": np.mean(p) / max(np.mean(r), 1e-6),
        "std_pred_length": np.std(p),
        "std_ref_length": np.std(r),
    }


# ================== BERTSCORE ==================

def compute_bertscore(preds: list[str], refs: list[str], lang: str) -> dict[str, float]:
    if not COMPUTE_BERT:
        return {"bertscore_f1": 0.0, "bertscore_precision": 0.0, "bertscore_recall": 0.0}

    preds = [str(p or "") for p in preds]
    refs = [str(r or "") for r in refs]

    if lang == "en":
        model_type = "microsoft/deberta-xlarge-mnli"
    else:
        model_type = "xlm-roberta-base"

    P, R, F1 = bert_score(
        preds,
        refs,
        model_type=model_type,
        lang=lang if lang == "en" else None,
        device=DEVICE,
        batch_size=BERT_BATCH_SIZE,
        verbose=False
    )

    return {
        "bertscore_precision": P.mean().item() * 100,
        "bertscore_recall": R.mean().item() * 100,
        "bertscore_f1": F1.mean().item() * 100,
    }


# ================== CORE EVAL ==================

def evaluate_task(preds: list[str], refs: list[str], lang: str, name: str) -> dict[str, float]:
    print(f"Evaluating: {name}")

    results = {
        "bleu": compute_bleu(preds, refs),
        "chrf": compute_chrf(preds, refs),
        "meteor": compute_meteor(preds, refs),
    }

    results.update(compute_length_stats(preds, refs))
    results.update(compute_bertscore(preds, refs, lang))

    print(
        f"    BLEU={results['bleu']:.2f} | chrF={results['chrf']:.2f} | "
        f"METEOR={results['meteor']:.2f} | BERT-F1={results['bertscore_f1']:.2f}"
    )

    return results


# ================== MODEL CONFIGS ==================

@dataclass
class TaskConfig:
    pred_col: str
    ref_col: str
    lang: str
    name: str


NLLB_TASKS = [
    TaskConfig("nllb_uz_to_uz", "uz_north_synthetic_ref", "other", "Uzbek normalization"),
    TaskConfig("nllb_uz_to_en_direct", "en_source", "en", "Direct translation"),
    TaskConfig("nllb_uz_to_en_pivot", "en_source", "en", "Pivot translation"),
    TaskConfig("nllb_uz_to_en_ensemble", "en_source", "en", "Ensemble translation"),
]

M2M_TASKS = [
    TaskConfig("pred_uzn_m2m", "uz_north_synthetic_ref", "other", "Uzbek normalization"),
    TaskConfig("pred_eng_m2m", "en_source", "en", "English translation"),
]


# ================== EVALUATION ==================

def print_comparison_table(all_results: dict[str, dict]):
    print("\n" + "=" * 100)
    print("FINAL COMPARISON TABLE")
    print("=" * 100)

    for model_name, results in all_results.items():
        print(f"\nModel: {model_name}")

        for task_name, metrics in results.items():
            print(f"{task_name}")

            line = (
                f"    BLEU: {metrics['bleu']:.2f} | "
                f"chrF: {metrics['chrf']:.2f} | "
                f"METEOR: {metrics['meteor']:.2f}"
            )

            if "bertscore_f1" in metrics:
                line += f" | BERT-F1: {metrics['bertscore_f1']:.2f}"

            print(line)


def evaluate_model(model_name: str, path: str):
    print(f"\nLoading: {path}")
    df = pl.read_csv(path)

    if "split" in df.columns:
        df = df.filter(pl.col("split") == "validation") 
        print(f"Using validation split: {df.height} samples")

    tasks = NLLB_TASKS if "nllb" in model_name.lower() else M2M_TASKS

    results = {}

    for task in tasks:
        if task.pred_col not in df.columns:
            continue

        preds = df[task.pred_col].to_list()
        refs = df[task.ref_col].to_list()

        results[task.name] = evaluate_task(preds, refs, task.lang, task.name)

    return results, df


# ================== MLFLOW ==================

def log_to_mlflow(model_name: str, results: dict[str, dict], df: pl.DataFrame, filename: str):
    with mlflow.start_run(run_name=f"Eval_{model_name}"):
        mlflow.log_param("model", model_name)
        mlflow.log_param("samples", df.height)
        mlflow.log_param("file", filename)

        for task, metrics in results.items():
            prefix = task.replace(" ", "_").lower()
            for k, v in metrics.items():
                mlflow.log_metric(f"{prefix}_{k}", v)

        mlflow.log_artifact(filename)


# ================== RUN ==================

def main():
    print("Starting evaluation")
    print("Device:", DEVICE)
    print("BERTScore:", COMPUTE_BERT)

    mlflow.set_experiment(EXPERIMENT_NAME)

    all_results = {}

    for model_name, path in FILES.items():
        results, df = evaluate_model(model_name, path)
        log_to_mlflow(model_name, results, df, path)
        all_results[model_name] = results
    print_comparison_table(all_results)

    print("\nDONE")


if __name__ == "__main__":
    main()

Starting evaluation
Device: cuda
BERTScore: True

Loading: /kaggle/input/predict1/predictions_model_1.csv
Using validation split: 516 samples
Evaluating: Uzbek normalization


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


    BLEU=14.75 | chrF=43.84 | METEOR=16.04 | BERT-F1=86.65
Evaluating: Direct translation


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


    BLEU=8.80 | chrF=38.27 | METEOR=27.10 | BERT-F1=60.29
Evaluating: Pivot translation


...

    BLEU=24.33 | chrF=39.82 | METEOR=28.94 | BERT-F1=64.09

Loading: /kaggle/input/pred2/predictions_model_2.csv
Using validation split: 516 samples
Evaluating: Uzbek normalization


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


    BLEU=2.71 | chrF=10.01 | METEOR=3.04 | BERT-F1=74.03
Evaluating: English translation


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


    BLEU=10.28 | chrF=18.59 | METEOR=16.80 | BERT-F1=46.40

FINAL COMPARISON TABLE

Model: NLLB-200
Uzbek normalization
    BLEU: 14.75 | chrF: 43.84 | METEOR: 16.04 | BERT-F1: 86.65
Direct translation
    BLEU: 8.80 | chrF: 38.27 | METEOR: 27.10 | BERT-F1: 60.29
Pivot translation
    BLEU: 24.46 | chrF: 34.92 | METEOR: 28.74 | BERT-F1: 64.16
Ensemble translation
    BLEU: 24.33 | chrF: 39.82 | METEOR: 28.94 | BERT-F1: 64.09

Model: M2M100
Uzbek normalization
    BLEU: 2.71 | chrF: 10.01 | METEOR: 3.04 | BERT-F1: 74.03
English translation
    BLEU: 10.28 | chrF: 18.59 | METEOR: 16.80 | BERT-F1: 46.40

DONE


In [36]:
pred1

,id,en_source,uz_south_source,uz_north_synthetic_ref,split,nllb_uz_to_uz,nllb_uz_to_en_direct,nllb_uz_to_en_pivot,nllb_uz_to_en_ensemble
0,1861,"Meanwhile, Mohammad Talib Shahid, head of publ...",شونده ی بیرحالده، هرات ساغلیق نی سقلش رئیسی مح...,Ayni paytda Hirotdagi sog'liqni saqlash boshqa...,validation,"Shu bilan birga, Heratning sog'liqni saqlash b...","In the meantime, Herat's head of health affair...","At the same time, Herat's head of health conce...","At the same time, Herat's head of health conce..."
1,59,This is because foreigners pursue their own go...,مقاومت مردمی اۉزیگه ته یه نیشی و تۉلیق مستقللی...,Chunki xorijliklar har qanday holatda ham o‘z ...,validation,"Odamlar qarshilik ko'rsatishga qarshi bo'lib, ...",resistance of the people to the revolution is ...,People stand up against opposition and act dec...,"People are resistant to resistance, and they'r..."
2,1308,According to the provincial governor's media o...,ولایت مطبوعات بۉلیمی نینگ خبریگه کۉره ، او ولا...,Viloyat gubernatori matbuot xizmati xabariga k...,validation,"O'zbekiston viloyati matbuot xizmati, viloyat ...","He was interviewed by the provincial press, an...","press service of Uzbekistan, the governor of t...","press service of Uzbekistan, the governor of t..."
3,1480,She also called for the financial cooperation ...,شونینگدیک او، یاردم بیرووچی نهاد لردن بدخشان د...,"U, shuningdek, Badaxshondagi qizlar va ayollar...",validation,"Shuningdek O, Yardam Bayruchi tashkiloti qizla...","Shuningdeek O, Yardem Beroochy Institute, has ...",He also demanded that the Yardam Bayruchi orga...,She also demands that the Yardami Bayruchi org...
4,2323,"""The child has been sick for 20 days.","""کسل بولگنی دن ۲۰ کون بوله دی.",“Bola 20 kundan beri kasal.,validation,"""Ksell Bolgunni kuni 20 kun bo'lib o'tdi.","""Ksell Bolgny did not bow for 20 days.","""It's been 20 days since Kessel's birthday.","""It's been 20 days since Castle's Birthday."
...,...,...,...,...,...,...,...,...,...
2574,822,"Simultaneously, a source from the institute in...",عین حالده، بو انستیتوتده گی بیر منبع نینگ ۸صبح...,"Shu bilan birga, institutdagi manbaning Hasht-...",test,"Shu bilan birga, bu institutda bir manba bor b...","At the same time, Bo Institute has a resource ...","At the same time, there is a source in this in...","At the same time, there is a source in this in..."
2575,1032,"In 2023, Transparency International assessed a...",شفّافلیک تشکیلاتی ۲۰۲۳- ییلده دنیا نینگ ۱۸۰ ته...,2023 yilda Transparency International dunyo bo...,test,2023-yilda Jahonning 180 yoshli davlatidagi ma...,"In 2023, when the world numbered 180, Sharif c...",In 2023 he reviewed and criticized the adminis...,"In 2023, he reviewed and criticized the admini..."
2576,902,Richard Bennett Urges Pakistan to Halt the Dep...,ریچارد بنت ینه بیر بار پاکستان دن افغانستانلیک...,Richard Bennet Pokistonni afg‘on qochqinlarini...,test,Richard Bennet bir marta Pokiston kunida Afg'o...,Richard Bennett once again in Pakistan on the ...,Richard Bennett once supported the draft law o...,Richard Bennett once backed the draft law on t...
2577,2423,According to the statistics provided by the mi...,اوشبو وزیرلیک آرقه لی ارائه بولگن آمار اساسیده...,Vazirlik tomonidan taqdim etilgan statistik ma...,test,Oshbo vazirining tartib-taomilini ko'rib chiqi...,Oshbu ministers have been orderly presenting b...,Based on statistics on the procedure of the As...,Based on statistics on the review of the proce...
